In [ ]:
# Logic to calculate occupiedSeats and availableSeats

x=2                         #number of seats with sensors. Could probably get this from the multiplexer channel scan.
sensorlist=list()

for i in range (x):         #Create a list with length x and with "NoN" as dummy values.
    sensorlist.append("NoN") #maybe skip this loop and just append 
    print(sensorlist)
print("initialized list:",sensorlist)

prox_first = 2300           #dummy value for testing
prox_second= 65000          #dummy value for testing
totalSeats=2                #totalSeats is hardcoded to "2" for our prototype since we only have two sensors per carriage.
#prox_first = get_proximity(sensor_prox_first)
#prox_second  = get_proximity(sensor_prox_second)

if prox_first <=2600:       #Find a better way to loop over sensors and populate the list?
    sensorlist[0] = False   #Populates a list with 2 elements (index 0 is for first sensor and index 1 is for second sensor) 
else: sensorlist[0]=True    #Values are either False or True and I have used arbitrarily chosen values as conditions.
                            #True if sensor value is greater than 2600, meaning the seat is occupied.
if prox_second <=2600:      #False if the sensor value is eqyualt to or less than 2600, menaning the seat is not occupied.
    sensorlist[1]=False
else: sensorlist[1]=True

print("populated list:", sensorlist)
occupiedSeats = sensorlist.count(True)
availableSeats=totalSeats - occupiedSeats
print("Occupied seats: ", occupiedSeats)
print("Available seats: ", availableSeats)

In [1]:
import sys

for i in range(1, len(sys.argv)):                                           # check argv string
    print('Id set to:', i, 'value:', sys.argv[i])
    id = sys.argv[i]


id=sys.argv[0]
print("id used: ", id)


Id set to: 1 value: --ip=127.0.0.1
Id set to: 2 value: --stdin=9011
Id set to: 3 value: --control=9009
Id set to: 4 value: --hb=9008
Id set to: 5 value: --Session.signature_scheme="hmac-sha256"
Id set to: 6 value: --Session.key=b"115aea9d-439d-4cfb-88a0-97a66da73a90"
Id set to: 7 value: --shell=9010
Id set to: 8 value: --transport="tcp"
Id set to: 9 value: --iopub=9012
Id set to: 10 value: --f=c:\Users\jespe\AppData\Roaming\jupyter\runtime\kernel-v2-28112bCsebNHs5o1G.json
id used:  C:\Users\jespe\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py


In [2]:
print("Please provide a carriage id as the first argument when running this script.  \n Example: 'Scriptname.py, ID'")



Please provide a carriage id as the first argument when running this script.  
 Example: 'Scriptname.py, ID'


In [ ]:
#Skanna sensorer och rapportera till en list, sensorlist 
import board
import adafruit_vcnl4010
import adafruit_tca9548a

i2c= board.I2C(board.SCL, board.SDA)  
tca = adafruit_tca9548a.TCA9548A(i2c)
for channel in range(8):                                    # Scan the multiplexer for sensors with addresses. Copied form tutorial
    if tca[channel].try_lock():                             # Channels are numbered 0-7
        print("Channel {}:".format(channel), end="")
        addresses = tca[channel].scan()
        print([hex(address) for address in addresses if address != 0x70])
        tca[channel].unlock()
        
        
print(addresses)





In [ ]:
for channel in range(8):                                    # Scan the multiplexer for sensors with addresses. Copied form tutorial
    if tca[channel].try_lock():                             # Channels are numbered 0-7
        print("Channel {}:".format(channel), end="")
        addresses = tca[channel].scan()
        print([hex(address) for address in addresses if address != 0x70])
        for address in addresses:
            if address !=0x70:
                sensorlist.append(address)
        print(sensorlist)
        tca[channel].unlock()

totalSeats= len(sensorlist)
print("Total no. of detected seats/sensors: ", totalSeats)    

In [ ]:
for channel in range(8):                                                             # Scan the multiplexer for sensors with addresses. Copied form tutorial
    if tca[channel].try_lock():                                                      # Channels are numbered 0-7
        print("Channel {}:".format(channel), end="")
        addresses = tca[channel].scan()
        print([hex(address) for address in addresses if address != 0x70])
        for address in addresses:                                                     #Stores all detected values (except 112/0x70) in a separate list, sensorlist
            if address !=0x70:
                sensorlist.append(address)
        print(sensorlist)
        tca[channel].unlock()

totalSeats= len(sensorlist)
print("Total no. of detected seats/sensors: ", totalSeats)    

In [ ]:
import adafruit_vcnl4010
import adafruit_tca9548a
import board

i2c= board.I2C(board.SCL, board.SDA)
tca = adafruit_tca9548a.TCA9548A(i2c) #Create an instance to the multiplexor
sensor_prox_first = adafruit_vcnl4010.VCNL4010(tca[1]) #Hardcoded, I checked and saw that the first sensor is on channel 1
sensor_prox_second = adafruit_vcnl4010.VCNL4010(tca[6]) #Hardcoded, I checked and saw that the second sensor is on channel 2 (6?)

def get_proximity(sensor): #Create a method for getting proximity data from a sensor.
	proximity = sensor.proximity
	print('Proximity: {0}'.format(proximity))
	return proximity

if get_proximity(sensor_prox_first) <=2600:          #Find a better way to loop over sensors and populate the list?
    sensorlist[0] = False                            #Populates a list with 2 elements (index 0 is for first sensor and index 1 is for second sensor) 
else: sensorlist[0]=True                             #Values are either False or True and I have used arbitrarily chosen values as conditions.
                                                     #True if sensor value is greater than 2600, meaning the seat is occupied.
if get_proximity(sensor_prox_second) <=2600:         #False if the sensor value is eqyualt to or less than 2600, menaning the seat is not occupied.
    sensorlist[1]=False
else: sensorlist[1]=True

In [ ]:
#create a dictionary and convert it to json and publish (necessary or can I convert directly, using the varibales)
import json

carriage_status = {
  "id": "1",                        #Carriage ID. Hardcoded value
  "occupiedSeats": occupiedSeats,
  "availableSeats": availableSeats
}

carriage_json = json.dumps(carriage_status)

print(carriage_json)


#Publish format (JSON string)
#{
#   id: '{id}',
#   availableSeats: '{number}',
#   occupiedSeats: '{number}'
#}

In [ ]:
# Loop that publishes message
while True:
    prox_first = get_proximity(sensor_prox_first)
    prox_second = get_proximity(sensor_prox_second)
    #prox = get_proximity()
    
    carriage_status = {
        "id": "1",                        #Carriage ID. Hardcoded value
        "occupiedSeats": occupiedSeats,
        "availableSeats": availableSeats
    }
    carriage_json = json.dumps(carriage_status)
    payload = carriage_json
    client.publish(topic, str(payload))
    time.sleep(2.0)	# Set delay

In [ ]:
import time
import datetime
import board
import adafruit_vcnl4010
import adafruit_tca9548a
import json
import paho.mqtt.client as mqtt
import paho.mqtt.publish as publish
#import busio

broker = "83.226.147.68"
#broker = "tcp://83.226.147.68:1883"
topic = "train/carriage/sensors"                # doublecheck this

#init I2C bus and sensor from lab3 + multiplexer
i2c= board.I2C(board.SCL, board.SDA)
sensor_prox = adafruit_vcnl4010.VCNL4010(i2c)	# Proximity

############### multiplexer section ##################
tca = adafruit_tca9548a.TCA9548A(i2c) #Create an instance to the multiplexor

for channel in range(8): #This is just to scan the multiplexor for sensors with addresses. Copied form tutorial
    if tca[channel].try_lock():
        print("Channel {}:".format(channel), end="")
        addresses = tca[channel].scan()
        print([hex(address) for address in addresses if address != 0x70])
        tca[channel].unlock()

############### Sensor section ##################
sensor_prox_first = adafruit_vcnl4010.VCNL4010(tca[1]) #Hardcoded, I checked and saw that the first sensor is on channel 1
sensor_prox_second = adafruit_vcnl4010.VCNL4010(tca[6]) #Hardcoded, I checked and saw that the second sensor is on channel 2 (6?)

def get_proximity(sensor): #Create a method for getting proximity data from a sensor.
	proximity = sensor.proximity
	print('Proximity: {0}'.format(proximity))
	return proximity

############### MQTT section ##################

# when connecting to mqtt do this;
def on_connect(client, userdata, flags, rc):
	if rc==0:
		print("Connection established. Code: "+str(rc))
	else:
		print("Connection failed. Code: " + str(rc))
		
def on_publish(client, userdata, mid):
    print("Published: " + str(mid))
	
def on_disconnect(client, userdata, rc):
	if rc != 0:
		print ("Unexpected disonnection. Code: ", str(rc))
	else:
		print("Disconnected. Code: " + str(rc))
	
def on_log(client, userdata, level, buf):		# Message is in buf
    print("MQTT Log: " + str(buf))

# Connect functions for MQTT
client = mqtt.Client()
client.on_connect = on_connect
client.on_disconnect = on_disconnect
client.on_publish = on_publish
client.on_log = on_log

#Connect to MQTT 
print("Attempting to connect to broker " + broker)
client.connect(broker)	# Broker address, port and keepalive (maximum period in seconds allowed between communications with the broker)
client.loop_start()

sensordata_list =list()
while True: # Loop and read proximity from sensor one and sensor two
    
    prox_first = get_proximity(sensor_prox_first)
    if prox_first <2600:
        prox_first_bool=False
    elif prox_first >= 2600:
        prox_first_bool=True
    sensordata_list[0]    
    prox_second = get_proximity(sensor_prox_second)
    if prox_second <2600:
        prox_second_bool =False
    elif prox_second >= 2600:
        prox_second_bool=True
    
    carriage_status = {                   #create a dict to contain values
        "id": "1",                        #Carriage ID. Hardcoded value
        "occupiedSeats": occupiedSeats,     
        "availableSeats": availableSeats
    }
    carriage_json = json.dumps(carriage_status) #convert dict to json string
    payload=carriage_json
    client.publish(topic, json(payload))
    time.sleep(1.0)



In [ ]:
while True:
    prox_first = get_proximity(sensor_prox_first)
    prox_second = get_proximity(sensor_prox_second)
    #prox = get_proximity()
    
    carriage_status = {                   #create a dict to contain values
        "id": "1",                        #Carriage ID. Hardcoded value
        "occupiedSeats": occupiedSeats,     
        "availableSeats": availableSeats
    }
    carriage_json = json.dumps(carriage_status) #convert to json string
    payload = carriage_json
    client.publish(topic, str(payload))
    time.sleep(2.0)	# Set delay